In [1]:
import joblib
from fastapi import FastAPI
import pandas as pd
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse
import threading
import uvicorn
from fastapi import HTTPException

# Load the trained model using joblib
def load_model():
    try:
        model = joblib.load("E:/GitHub/Bati-Bank_Credit-Scoring-Model/Random_Forest_model.pkl")
        # Check if model has predict method
        if not hasattr(model, 'predict'):
            raise Exception("The loaded object is not a valid model. Please check the saved model.")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        raise HTTPException(status_code=500, detail=str(e))

# Initialize FastAPI app
app = FastAPI()

# Load model at app startup
model = load_model()

# Allow CORS for frontend interaction
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Define request body format
class InputData(BaseModel):
    Transaction_Count: int
    Total_Transaction_Amount: float
    Average_Transaction_Amount_woe: float
    TransactionMonth_woe: float
    TransactionDay_woe: float
    ProductCategory_financial_services_woe: float
    ProviderId_woe: float
    Transaction_Count_woe: float
    Value: float

@app.post("/predict")
def predict(data: InputData):
    try:
        # Convert input data to DataFrame
        df = pd.DataFrame([data.dict()])

        # Debugging: Print input data
        print("Input data:", df)
        
        # Make prediction
        prediction = model.predict(df)[0]
        probability = model.predict_proba(df)[0][1]  # Probability of fraud
        
        # Debugging: Print prediction and probability
        print(f"Prediction: {prediction}, Probability: {probability}")
        
        return {"prediction": int(prediction), "risk_probability": float(probability)}
    
    except Exception as e:
        print(f"Error: {e}")
        raise HTTPException(status_code=500, detail=str(e))

# HTML and CSS for GUI
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Credit Scoring Prediction</title>
    <style>
        body { font-family: Arial, sans-serif; text-align: center; }
        form { display: inline-block; text-align: left; }
        input { margin: 5px; padding: 8px; }
        button { padding: 10px; background-color: blue; color: white; }
    </style>
</head>
<body>
    <h1>Credit Scoring Prediction</h1>
    <form id="predictForm">
        <input type="text" id="Transaction_Count" placeholder="Transaction Count" required><br>
        <input type="text" id="Total_Transaction_Amount" placeholder="Total Transaction Amount" required><br>
        <input type="text" id="Average_Transaction_Amount_woe" placeholder="Average Transaction Amount " required><br>
        <input type="text" id="TransactionMonth_woe" placeholder="Transaction Month" required><br>
        <input type="text" id="TransactionDay_woe" placeholder="Transaction Day " required><br>
        <input type="text" id="ProductCategory_financial_services_woe" placeholder="Product Category " required><br>
        <input type="text" id="ProviderId_woe" placeholder="Provider ID " required><br>
        <input type="text" id="Transaction_Count_woe" placeholder="Transaction Count " required><br>
        <input type="text" id="Value" placeholder="Value" required><br>
        <button type="submit">Predict</button>
    </form>
    <p id="result"></p>
    <script>
        document.getElementById("predictForm").addEventListener("submit", async function(event) {
            event.preventDefault();
            let data = {
                Transaction_Count: parseInt(document.getElementById("Transaction_Count").value),
                Total_Transaction_Amount: parseFloat(document.getElementById("Total_Transaction_Amount").value),
                Average_Transaction_Amount_woe: parseFloat(document.getElementById("Average_Transaction_Amount_woe").value),
                TransactionMonth_woe: parseFloat(document.getElementById("TransactionMonth_woe").value),
                TransactionDay_woe: parseFloat(document.getElementById("TransactionDay_woe").value),
                ProductCategory_financial_services_woe: parseFloat(document.getElementById("ProductCategory_financial_services_woe").value),
                ProviderId_woe: parseFloat(document.getElementById("ProviderId_woe").value),
                Transaction_Count_woe: parseFloat(document.getElementById("Transaction_Count_woe").value),
                Value: parseFloat(document.getElementById("Value").value)
            };
            let response = await fetch("/predict", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify(data)
            });
            let result = await response.json();
            document.getElementById("result").innerText = `Prediction: ${result.prediction}, Risk Probability: ${result.risk_probability}`;
        });
    </script>
</body>
</html>
"""

@app.get("/", response_class=HTMLResponse)
def serve_gui():
    return HTMLResponse(content=html_content, status_code=200)

# Function to run FastAPI app in a separate thread
def run_api():
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Run FastAPI in a background thread to avoid event loop conflicts
if __name__ == "__main__":
    threading.Thread(target=run_api, daemon=True).start()
    # Keep the main program running to keep the server alive
    input("Press Enter to stop the server...\n")


c:\Users\Abdilala\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Abdilala\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
INFO:     Started server process [9328]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+

INFO:     127.0.0.1:63208 - "GET / HTTP/1.1" 200 OK
